In [ ]:
from google.colab import drive                                # 구글 드라이버와 연동
drive.mount("/content/drive")

import pandas as pd                                            # 데이터프레임 형성
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel             # 여러 문장 유사도 계산

file = "/content/drive/MyDrive/NLP/movies_metadata.csv"        # 활용 데이터

# 데이터프레임 생성
df = pd.read_csv(file)                                         # csv 파일 읽기
pd.options.display.max_columns = None                          # 모든 컬럼 보이기
print(f'상위 5개의 데이터: \n{df.head()}')                     # 부분 데이터 출력

# 컬럼의 결측치 확인
overview_null = df['overview'].isnull().sum()
title_null = df["title"].isnull().sum()
print(f'줄거리 컬럼의 결측치 개수: {overview_null}')
print(f'제목 컬럼의 결측치 개수: {title_null}')

# 컬럼의 결측치 제거
df.dropna(subset = ["overview", "title"], inplace = True)

# 불용어 제거(영어)
tfidf = TfidfVectorizer(stop_words = 'english')

# 단어 사전 생성
data = df.iloc[:20000, :]                                        # 부분 활용 데이터(2만개)
tfidf.fit(data.overview)                                        # 단어 사전 생성(줄거리)
vocab = tfidf.vocabulary_
print(f'2만 개 영화 줄거리 단어의 개수: {len(vocab)}')          # 단어 사전 개수 확인

matrix = tfidf.transform(data.overview).toarray()               # 각 문장의 행렬 변환
print(f'행렬 모양: {matrix.shape}')                             # 행렬 모양 확인

# 인덱스 재설정
reset_data = data.reset_index(drop = True)

# 유사도 측정
similar_matrix = linear_kernel(matrix, matrix)
print(f'유사도 측정: \n{similar_matrix}')

# 제목 순서화(인덱스) + 중복 제목 제거
extract_index = pd.Series(data = reset_data.index, index = reset_data.title).drop_duplicates()

# 영화 추천 함수생성
def recommend(title, similar_matrix):

    # 영화의 제목을 이용해 해당 영화의 인덱스 확인
    idx_title = extract_index[title]

    # 모든 영화와 특정 영화와의 유사도 구하기
    similar_score = [(idx, score) for idx, score in enumerate(similar_matrix[idx_title])]
    print(f'유사도 측정 결과: \n{similar_score}')

    # 유사도에 따른 정렬
    sorted_similar_score = sorted(similar_score, key = lambda x:x[1], reverse = True)
    print(sorted_similar_score)

    # 유사도가 가장 큰 10개의 영화의 인덱스 확인
    similar_scores = sorted_similar_score[1:11]
    print(f'유사도가 높은 10개의 영화: \n{similar_scores}')
    movie_index = [data[0] for data in similar_scores]
    print(f'유사도가 높은 10개 영화의 인덱스: \n{movie_index}')

    # 유사도가 가장 큰 10개의 영화 제목 확인
    movie_title = data['title'].iloc[movie_index]

    return movie_title

# 함수 테스트
recommended_movies = recommend("Jumanji", similar_matrix)
print(f'Jumanji와 유사도가 높은 10개의 영화: \n{recommended_movies}')